In [9]:
!pip install tensorflow_datasets

     |████████████████████████████████| 3.9 MB 4.2 MB/s eta 0:00:01
     |████████████████████████████████| 48 kB 11.3 MB/s eta 0:00:01
     |████████████████████████████████| 198 kB 27.0 MB/s eta 0:00:01
  Created wheel for promise: filename=promise-2.3-py3-none-any.whl size=21494 sha256=aa175df5262909d46312e5e27ed80fa03c84dec7fe9a35a4d397d66b8e8707b7
  Stored in directory: /Users/sathishrajendiran/Library/Caches/pip/wheels/54/aa/01/724885182f93150035a2a91bce34a12877e8067a97baaf5dc8
Successfully built promise


In [1]:
# SINGLE LAYER AND MULTI LAYER NETWORKS FOR MNIST
# BASED ON CODE FROM TENSORFLOW TUTORIAL

import tensorflow as tf

In [3]:
# MY TEST TO MAKE SURE TF IS WORKING

# hello = tf.constant('Hello, TensorFlow!')
# sess = tf.Session()
# print(sess.run(hello))

tf.executing_eagerly()
with tf.compat.v1.Session() as sess:
    hello = tf.constant('Hello, TensorFlow!')
    print(sess.run(hello))

b'Hello, TensorFlow!'


In [31]:
# OBTAIN DATA
# DATA WITHIN TF TUTORIAL HAS BEEN PREPROCESSED

# from tensorflow.examples.tutorials.mnist import input_data
# mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)


import tensorflow_datasets


mnist = tensorflow_datasets.load('mnist')

# import input_data
# mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)


# from tensorflow.examples.tutorials.mnist import input_data
# mnist = input_data.read_data_sets("MNIST_data", one_hot=True)

In [32]:
mnist

{'test': <PrefetchDataset shapes: {image: (28, 28, 1), label: ()}, types: {image: tf.uint8, label: tf.int64}>,
 'train': <PrefetchDataset shapes: {image: (28, 28, 1), label: ()}, types: {image: tf.uint8, label: tf.int64}>}

In [33]:
# MODEL
# CREATE PLACEHOLDER VARIABLES FOR OPERATION MANIPULATION
# THE 784 MATCHES THE VECTOR SIZE OF THE MNIST IMAGES - 28*28

import tensorflow.compat.v1 as tf
tf.disable_v2_behavior() 

x = tf.placeholder(tf.float32, [None, 784])

In [34]:
# MODEL
# CREATE WEIGHTS & BIASES VARIABLES
# IN TF, OUR MODEL PARAMETERS ARE OFTEN MANAGED AS VARIABLES

W = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10]))

In [35]:
# MODEL
# CREATE MODEL - USES SOFTMAX AS THE ACTIVATION FUNCTION
# REMEMBER GOAL FOR ACTIVATION FUNCTION IS TO "SHAPE" THE 
# OUTPUT INTO A PROBABILITY DISTRO OVER THE 10 CLASSES

y = tf.nn.softmax(tf.matmul(x, W) + b)


In [36]:
# MODEL
# CREATE PREDICTED VARIABLE Y-HAT
# AND USE CROSS ENTROPY TO DETERMINE LOSS
# CROSS ENTROPY - HOW INEFFICIENT ARE OUR PREDICTIONS?

y_ = tf.placeholder(tf.float32, [None, 10])
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))


In [37]:
# MODEL
# TRAIN USING GRADIENT DESCENT
# LEARNING RATE AT MIDPOINT - .5 - MAKE SMALL ADJUSTMENTS TO MINIMIZE COST

train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

In [38]:
print(tf.__version__)

2.5.0


In [43]:
import numpy as np

def next_batch(num, data, labels):
    '''
    Return a total of `num` random samples and labels. 
    '''
    idx = np.arange(0 , len(data))
    np.random.shuffle(idx)
    idx = idx[:num]
    data_shuffle = [data[ i] for i in idx]
    labels_shuffle = [labels[ i] for i in idx]

    return np.asarray(data_shuffle), np.asarray(labels_shuffle)


In [44]:
# MODEL - RUN

sess = tf.InteractiveSession()
tf.global_variables_initializer().run()
for _ in range(10000):
  batch_xs, batch_ys = mnist['train'].next_batch(100)
  sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys})

AttributeError: 'PrefetchDataset' object has no attribute 'next_batch'

In [51]:
print (mnist['train'].__class__)

<class 'tensorflow.python.data.ops.dataset_ops.PrefetchDataset'>


In [10]:
# EVALUATE MODEL

correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print(sess.run(accuracy, feed_dict={x: mnist.test.images, y_: mnist.test.labels}))


0.9203


## Block Two for MNIST on TensorFlow

The following code offers an alternative approach to the MNIST classification problem.
Using two convolution layers, we can significantly improve upon the original model and explore some of the options for a deep neural network.


In [11]:
# WEIGHT INITIALIZATION

def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)

def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)

In [12]:
# CREATE CONVOLUTION AND POOLING LAYERS

def conv2d(x, W):
  return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
  return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')

In [13]:
# FIRST CONVOLUTION LAYER

W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])

x_image = tf.reshape(x, [-1, 28, 28, 1]) # BASE IMAGE SIZE OF 28 * 28

h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)  # RESULTING IMAGE SIZE IS 14 * 14


In [14]:
# SECOND CONOLUTION LAYER 
# MORE THAN ONE LAYER?  DEEP LEARNING

W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

In [15]:
# FULLY CONNECTED LAYER - BEFORE OUTPUT

W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)  # ADD THE RECTIFIED LINEAR UNIT


In [16]:
# DROP LAYER - REDUCE OVERFITTING

keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

In [17]:
# LAST LAYER - OUTPUT

W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2

In [18]:
# RUN THE MODEL

cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_conv))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

with tf.Session() as sess:
  sess.run(tf.global_variables_initializer())
  for i in range(10000):
    batch = mnist.train.next_batch(50)
    if i % 100 == 0:
      train_accuracy = accuracy.eval(feed_dict={
          x: batch[0], y_: batch[1], keep_prob: 1.0})
      print('step %d, training accuracy %g' % (i, train_accuracy))
    train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})

  print('test accuracy %g' % accuracy.eval(feed_dict={
      x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))

step 0, training accuracy 0.04
step 100, training accuracy 0.86
step 200, training accuracy 0.9
step 300, training accuracy 0.9
step 400, training accuracy 0.98
step 500, training accuracy 0.82
step 600, training accuracy 0.98
step 700, training accuracy 0.96
step 800, training accuracy 0.96
step 900, training accuracy 0.98
step 1000, training accuracy 0.92
step 1100, training accuracy 0.94
step 1200, training accuracy 0.94
step 1300, training accuracy 0.92
step 1400, training accuracy 1
step 1500, training accuracy 0.92
step 1600, training accuracy 0.98
step 1700, training accuracy 0.94
step 1800, training accuracy 0.96
step 1900, training accuracy 0.98
step 2000, training accuracy 0.98
step 2100, training accuracy 0.98
step 2200, training accuracy 0.98
step 2300, training accuracy 0.96
step 2400, training accuracy 0.96
step 2500, training accuracy 1
step 2600, training accuracy 0.98
step 2700, training accuracy 1
step 2800, training accuracy 1
step 2900, training accuracy 0.94
step 3

In [19]:
# System Information
import platform
import pandas as pd
print('Python is ' + platform.python_version())

pd.show_versions(as_json=False)

Python is 3.5.3

INSTALLED VERSIONS
------------------
commit: None
python: 3.5.3.final.0
python-bits: 64
OS: Windows
OS-release: 10
machine: AMD64
processor: Intel64 Family 6 Model 158 Stepping 9, GenuineIntel
byteorder: little
LC_ALL: None
LANG: None
LOCALE: None.None

pandas: 0.20.1
pytest: 3.0.7
pip: 9.0.1
setuptools: 27.2.0
Cython: 0.25.2
numpy: 1.12.1
scipy: 0.19.1
xarray: None
IPython: 5.3.0
sphinx: 1.5.6
patsy: 0.4.1
dateutil: 2.6.0
pytz: 2017.2
blosc: None
bottleneck: 1.2.1
tables: 3.2.2
numexpr: 2.6.2
feather: None
matplotlib: 2.0.2
openpyxl: 2.4.7
xlrd: 1.0.0
xlwt: 1.2.0
xlsxwriter: 0.9.6
lxml: 3.7.3
bs4: 4.6.0
html5lib: 0.9999999
sqlalchemy: 1.1.9
pymysql: None
psycopg2: None
jinja2: 2.9.6
s3fs: None
pandas_gbq: None
pandas_datareader: None
